In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS,analyzer='word', 
                             binary=True,min_df=10, max_df=.04)
X_train = vectorizer.fit_transform(newsgroups_train.data)

X_train.toarray()
docs,words=X_train.nonzero() 
print(X_train.shape) #(11314, 10299)
print(len(docs)) #it is N=480590

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


(11314, 10299)
480590


In [2]:
import numpy as np
from tqdm import tqdm
M=11314 #amount of texts
W=10299 #amount of different words
N=480590#total amount of words in the corpus
K=20 #amount of tags
NITER=50


def lda(X_train,alpha,beta,NITER):
    X=X_train.toarray()
    docs,words=X.nonzero() #print(len(docs)) #it is N=480590
    z=[np.random.choice(K) for i in range(N)] #randomly assign tags to all words in the corpus
    
    n_dk=np.zeros(M*K).reshape(M,K) #amount of words in document d assigned to tag k
    n_kw=np.zeros(K*W).reshape(K,W) #amount of times word w was assigned to tag k
    n_k=np.zeros(K) #total amount of words assigned to tag k
    
    for doc, word, cur_z in zip(docs, words, z):
        n_dk[doc, cur_z] += 1
        n_kw[cur_z, word] += 1
        n_k[cur_z] += 1
        
    for tek_iter in tqdm(range(NITER)):
        for i in range(N):
            n_dk[docs[i],z[i]]-=1
            n_kw[z[i],words[i]]-=1
            n_k[z[i]]-=1
            p = (n_dk[docs[i], :] + alpha) * (n_kw[:, words[i]] + beta[words[i]]) / (n_k + beta.sum())
            z[i] = np.random.choice(np.arange(K), p=p / p.sum())
            
            n_dk[docs[i],z[i]]+=1
            n_kw[z[i],words[i]]+=1
            n_k[z[i]]+=1
    return  n_kw

alpha=np.ones(K)
beta=np.ones(W)


print(alpha[0:10])

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [3]:
n_kw=lda(X_train,alpha,beta,NITER)
#print(n_kw)

100%|██████████| 50/50 [23:19<00:00, 27.99s/it]


In [4]:
n_kw_sorted=np.argsort(n_kw, axis=1)
#print(vectorizer.vocabulary_.get('car'))
#print(type(vectorizer.vocabulary_))

my_dict={}
for item in vectorizer.vocabulary_:
    my_dict[vectorizer.vocabulary_[item]]=item

for k in range(K):
    print("tag=",k,end=" ")
    for j in range(10):
        print(my_dict[n_kw_sorted[k,W-1-j]],end=" ")
    print("\n")

tag= 0 soon gordon medical banks pitt usually surrender disease cause geb 

tag= 1 knows sorry stuff guess similar deleted interesting current answer couldn 

tag= 2 bike ride dod road left difference hand riding likely motorcycle 

tag= 3 gun law crime control guns laws weapons police self likely 

tag= 4 went told started days saw took came happened left asked 

tag= 5 appreciated info anybody posting thank net answer sorry wondering check 

tag= 6 space nasa research earth science university project low center orbit 

tag= 7 israel country war rights jews israeli american anti land military 

tag= 8 card pc computer hi memory disk video dos mac monitor 

tag= 9 jesus christian bible man christians religion word christ saying claim 

tag= 10 experience recently correct apparently couple current small add reading article 

tag= 11 game team games play season players league win 11 12 

tag= 12 advance wanted cheers david reading interesting small reply news open 

tag= 13 turkish today